### Data Ingestion

In [2]:
import os
%pwd

'c:\\Users\\ranji\\Projects\\Cruxit\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\ranji\\Projects\\Cruxit'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: Path
    local_data_file: Path
    unzip_dir: Path

In [25]:
from src.cruxit.constants import *
from src.cruxit.utils.common import read_yaml, create_directories

In [26]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [27]:
import os
import urllib.request as request
import zipfile
from src.cruxit.logging import logger
from src.cruxit.utils.common import get_size

In [28]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"File {filename} is downloaded!")
        else:
            logger.info(f"File already exists!")  

        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [30]:
config = ConfigurationManager()
data_ingestion_config = config.get_data_ingestion_config()
data_ingestion = DataIngestion(config=data_ingestion_config)
data_ingestion.download_file()
data_ingestion.extract_zip_file()


[2025-05-15 23:32:21,025: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-05-15 23:32:21,027: INFO: common: yaml file: params.yaml loaded successfully]
[2025-05-15 23:32:21,030: INFO: common: created directory at: artifacts]
[2025-05-15 23:32:21,032: INFO: common: created directory at: artifacts/data_ingestion]
[2025-05-15 23:32:21,915: INFO: 2175897188: File artifacts/data_ingestion/data.zip is downloaded!]
